# Question 2
### Working with ADQL

In [2]:
# Basic imports
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table

In [4]:
# Download the GAIA Data
from astroquery.gaia import Gaia

From the Gaia/2MASS Tables, we need:
- Gaia BP-RP
- Gaia absolute g
- 2Mass J-Ks
- 2Mass Apparent H

In [109]:
# Select relevant columns and join with the 2Mass table. Constrain ra/dec to
# a 1 degree circle from Ruprecht 147
query = """
SELECT gaia.source_id, gaia.ra, gaia.dec, gaia.parallax, gaia.phot_g_mean_mag, 
gaia.bp_rp, tmass.j_m, tmass.ks_m, tmass.h_m, tmass.ph_qual
FROM gaiadr3.gaia_source AS gaia
JOIN gaiadr3.tmass_psc_xsc_best_neighbour AS xmatch USING (source_id)
JOIN gaiadr3.tmass_psc_xsc_join AS xjoin USING (clean_tmass_psc_xsc_oid)
JOIN gaiadr1.tmass_original_valid AS tmass ON
   xjoin.original_psc_source_id = tmass.designation
WHERE phot_g_mean_mag < 14
   AND CONTAINS(
     POINT('ICRS', gaia.ra, gaia.dec),
     CIRCLE('ICRS', 289.074, -16.323, 1.0)
   ) = 1;
"""
job = Gaia.launch_job_async(query)
result = job.get_results()
df = result.to_pandas()
display(df.head())
print("Dataframe has shape", df.shape)


INFO: Query finished. [astroquery.utils.tap.core]


,source_id,ra,dec,parallax,phot_g_mean_mag,bp_rp,j_m,ks_m,h_m,ph_qual
0,4088892600474423808,288.225913,-15.744217,0.624552,11.983528,1.559996,10.017,9.218,9.410,AAA
1,4088893154546972672,288.271283,-15.709582,0.454480,12.048648,1.703223,9.941,9.005,9.188,AAA
2,4088893188906716032,288.254396,-15.713978,0.211160,12.808913,1.847858,10.526,9.503,9.733,AAA
3,4087815667506086656,288.512276,-16.874361,0.092825,12.800300,1.850592,10.501,9.492,9.710,AAA
4,4087815697559746304,288.520970,-16.874898,0.366892,11.617373,3.226758,8.277,7.032,7.314,AAA


Dataframe has shape (3637, 10)


In [110]:
# Manipulate column names to be easier to work with, and add absolute g magnitude
# Add an if statement to avoid errors when re-running this cell
if len(df.columns) == 10:
    df.columns = ["ID", "RA", "DEC", "parallax", "g_m", "bp_rp", "j_m", "ks_m", "h_m", "quality"]
    df["g_M"] = df["g_m"]+5*np.log10(df["parallax"]/1000)+5
display(df.head())

c:\Users\taase\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,ID,RA,DEC,parallax,g_m,bp_rp,j_m,ks_m,h_m,quality,g_M
0,4088892600474423808,288.225913,-15.744217,0.624552,11.983528,1.559996,10.017,9.218,9.410,AAA,0.961373
1,4088893154546972672,288.271283,-15.709582,0.454480,12.048648,1.703223,9.941,9.005,9.188,AAA,0.336221
2,4088893188906716032,288.254396,-15.713978,0.211160,12.808913,1.847858,10.526,9.503,9.733,AAA,-0.568030
3,4087815667506086656,288.512276,-16.874361,0.092825,12.800300,1.850592,10.501,9.492,9.710,AAA,-2.361372
4,4087815697559746304,288.520970,-16.874898,0.366892,11.617373,3.226758,8.277,7.032,7.314,AAA,-0.559937


The error in the log10 function here comes from stars with a non-positive parallax. 

From the shape of the dataframe, we can see that the original query yields **3,637 stars**

To see the stars with a poor quality, we can do a selection slice on the "quality" column:

In [126]:
df[np.isnan(df["parallax"])]

,ID,RA,DEC,parallax,g_m,bp_rp,j_m,ks_m,h_m,quality,g_M
74,4087821882312651008,288.525499,-16.827445,NaN,11.821259,0.800284,10.763,10.448,10.528,AAA,NaN
231,4087839508853170048,289.071281,-16.684002,NaN,10.886174,0.677984,9.795,9.540,9.564,AAA,NaN
410,4087854665809853824,289.104206,-16.533982,NaN,11.686994,1.011149,10.378,9.932,10.025,AAA,NaN
729,4088142188104169216,288.310096,-15.751787,NaN,13.163245,0.772991,11.582,11.249,11.319,AAA,NaN
1068,4087750968104080000,289.051417,-17.069334,NaN,12.869500,NaN,10.914,10.577,10.659,AAA,NaN
1069,4087750968116980096,289.051486,-17.069223,NaN,12.647315,0.844853,10.914,10.577,10.659,AAA,NaN
1532,4087794398827633920,288.663502,-17.104981,NaN,13.482010,0.915640,12.358,11.975,12.039,AAA,NaN
1572,4087799758947342592,288.836737,-17.065648,NaN,12.789598,0.906240,11.372,10.983,11.031,AAA,NaN
1610,4087805187773003136,288.711863,-16.956434,NaN,12.652823,NaN,10.691,10.422,10.474,AAA,NaN
2023,4183945281925731840,289.241655,-16.241196,NaN,11.623805,0.971126,10.362,9.963,10.026,AAA,NaN


In [134]:
df_lowqual = df[df["quality"] != "AAA"]
print(df_lowqual.shape)
df_bad_parallax = df[(df["parallax"] < 0) | (np.isnan(df["parallax"]))]
print(df_bad_parallax.shape)

(79, 11)
(23, 11)


From the quality cuts, we can see **79 stars** have a low 2Mass photometric quality and **23 stars** have a non-positive or NaN parallax

In [133]:
df_good = df[(df["quality"] == "AAA") & (df["parallax"] > 0)]
print(df_good.shape)

(3535, 11)


After the quality cuts, there are **3535 stars** in total